In [29]:
# !pip3 install pandas openpyxl plotly

In [109]:
import pandas as pd
import re
import json
from urllib.parse import parse_qs, urlparse, unquote
import plotly.express as px
import plotly.offline as pyo
pyo.init_notebook_mode(connected=True)

pd.set_option('display.max_columns', None)
pd.set_option('display.max_colwidth', None)

In [110]:
log_file_path = 'all_logs/spellbook.log'

In [111]:
def parse_log_entry(entry):
    # Pattern for the main log entry structure
    main_pattern = r'\[pid: (\d+)\|app: (\d+)\|req: (\d+)/(\d+)\] (\d+\.\d+\.\d+\.\d+) \(\) \{(\d+) vars in (\d+) bytes\} \[(.*?)\] (\w+) (.*?) => generated (\d+) bytes in (\d+) msecs(?: via (.*?))? \((.*?)\) (\d+) headers in (\d+) bytes \((\d+) switches on core (\d+)\)'
    main_match = re.match(main_pattern, entry)

    # Pattern for Elasticsearch warning
    es_warning_pattern = r'WARNING:elasticsearch:(.*?) \[status:(\d+) request:(.*?)s\]'
    es_warning_match = re.match(es_warning_pattern, entry)

    # Pattern for general warnings and errors
    general_warning_pattern = r'(.*?):(\d+): (.*?):\n\n(.*?)\n\n(.*?)\n'
    general_warning_match = re.match(general_warning_pattern, entry)

    # Pattern for OSError
    oserror_pattern = r'OSError: (.*)'
    oserror_match = re.match(oserror_pattern, entry)

    # Pattern for SIGPIPE errors
    sigpipe_pattern = r'(\w+ \w+  \d+ \d+:\d+:\d+ \d+) - SIGPIPE: (.*?) on request (.*?) \(ip (.*?)\) !!!'
    sigpipe_match = re.match(sigpipe_pattern, entry)

    if main_match:
        return {
            'type': 'main',
            'pid': int(main_match.group(1)),
            'app': int(main_match.group(2)),
            'req_id': int(main_match.group(3)),
            'req_total': int(main_match.group(4)),
            'client_ip': main_match.group(5),
            'vars_count': int(main_match.group(6)),
            'vars_size': int(main_match.group(7)),
            'timestamp': main_match.group(8),
            'method': main_match.group(9),
            'endpoint': main_match.group(10),
            'response_size': int(main_match.group(11)),
            'response_time': int(main_match.group(12)),
            'via': main_match.group(13) if main_match.group(13) else '',
            'http_version_status': main_match.group(14),
            'headers_count': int(main_match.group(15)),
            'headers_size': int(main_match.group(16)),
            'switches': int(main_match.group(17)),
            'core': int(main_match.group(18))
        }
    elif es_warning_match:
        return {
            'type': 'es_warning',
            'message': es_warning_match.group(1),
            'status': int(es_warning_match.group(2)),
            'request_time': float(es_warning_match.group(3))
        }
    elif general_warning_match:
        return {
            'type': 'general_warning',
            'file': general_warning_match.group(1),
            'line': int(general_warning_match.group(2)),
            'warning_type': general_warning_match.group(3),
            'message': general_warning_match.group(4),
            'details': general_warning_match.group(5)
        }
    elif oserror_match:
        return {
            'type': 'os_error',
            'message': oserror_match.group(1)
        }
    elif sigpipe_match:
        return {
            'type': 'sigpipe_error',
            'timestamp': sigpipe_match.group(1),
            'error_message': sigpipe_match.group(2),
            'request': sigpipe_match.group(3),
            'client_ip': sigpipe_match.group(4)
        }
    else:
        return None

## Loading the log file

In [112]:
with open(log_file_path, 'r') as file:
    log_entries = file.readlines()

parsed_entries = [parse_log_entry(entry) for entry in log_entries if parse_log_entry(entry) is not None]

df = pd.DataFrame(parsed_entries)

# Splitting between API and Web interfaces

In [113]:
df['endpoint'] = df['endpoint'].fillna('')
df_api = df[df['endpoint'].str.contains('/api/generate_document')]
df_web = df[df['endpoint'].str.contains('/download_generated_document')]

In [116]:
def parse_url(url):
    parsed_url = urlparse(url)
    params = parse_qs(parsed_url.query)
    
    # Extract the last directory of the URL path
    category = parsed_url.path.split('/')[-1]

    flat_params = {'category': category}
    for key, value in params.items():
        # Unquote the value to handle URL-encoded strings
        unquoted_value = unquote(value[0])
        
        # Try to load JSON data if present
        try:
            json_value = json.loads(unquoted_value)
            if isinstance(json_value, dict):
                # Flatten nested JSON
                for sub_key, sub_value in json_value.items():
                    flat_params[f"{key}.{sub_key}"] = sub_value
            else:
                flat_params[key] = json_value
        except json.JSONDecodeError:
            flat_params[key] = unquoted_value

    return flat_params

In [117]:
df_web_processed = pd.DataFrame(df_web['endpoint'].apply(parse_url).tolist()).drop_duplicates()
df_api_processed = pd.DataFrame(df_api['endpoint'].apply(parse_url).tolist()).drop_duplicates()

## Generating stats

In [124]:
print("Total documents:", len(df_web_processed['category']) + len(df_api_processed['category']))

Total documents: 120


In [118]:
t0 = df_api_processed['data.generation_date'].min()
t1 = df_api_processed['data.generation_date'].max()

print("Regarding API interface!")
print("Between",t0,"and",t1)
print()
display(df_api_processed['category'].value_counts().reset_index())


Regarding API interface!
Between 2024-07-01 and 2024-07-01



,category,count
0,BlueTeamInternal,87


In [120]:
df_web_processed['data.generation_date'] = df_web_processed['filename'].apply(lambda x: x.split(' ')[0])

t0 = df_web_processed['data.generation_date'].min()
t1 = df_web_processed['data.generation_date'].max()

print("Regarding Web interface!")
print("Between",t0,"and",t1)
print()
df_web_processed['category'].value_counts().reset_index()

Regarding Web interface!
Between 2024-07-01 and 2024-07-02



,category,count
0,BlueTeamInternal,7
1,General,5
2,CERT,5
3,BlueTeamSPO,4
4,SPO,4
5,NBS,3
6,RedTeam,2
7,BlueTeam,2
8,CERTFR,1
